# 🧠 Feature Selection and Classification (With & Without Optimization)

Dataset: **Breast Cancer Wisconsin (Diagnostic)** (30 numeric features, 569 rows) from `sklearn.datasets`.

This notebook:
1) Loads & prepares data
2) Trains baseline classifiers (no feature selection)
3) Applies **SelectKBest (ANOVA F-test)** and **RFE**
4) Retrains classifiers using selected features
5) Compares metrics (Accuracy, Precision, Recall, F1) & Confusion Matrices
6) Saves two models (`model_without_fs.pkl`, `model_with_fs.pkl`) as **pipelines**
7) (Optional) Use with provided `app.py` Streamlit app


In [ ]:
# Step 1: Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import joblib
import json as _json

RANDOM_STATE = 42


In [ ]:
# Step 2: Load & Inspect Data
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

print('Shape:', X.shape)
display(X.head())
print('\nClass distribution:')
print(y.value_counts())

In [ ]:
# Step 3: Handle Missing, Encode, Scale, Split
# (Dataset is clean & numeric already; no missing values or categoricals)
print('Missing values per column (should be 0):')
print(X.isna().sum().sum())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
print('Train:', X_train.shape, ' Test:', X_test.shape)

## Step 4A: Baseline Models (No Feature Selection)
We will train two classifiers using **all features**:
- Logistic Regression
- Random Forest

We'll wrap them in **Pipelines** with StandardScaler where appropriate.

In [ ]:
baseline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])
baseline_rf = Pipeline([
    ('clf', RandomForestClassifier(random_state=RANDOM_STATE))
])

baseline_lr.fit(X_train, y_train)
baseline_rf.fit(X_train, y_train)

def eval_model(model, X_tr, y_tr, X_te, y_te, title='Model'):
    y_pred = model.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    prec = precision_score(y_te, y_pred)
    rec = recall_score(y_te, y_pred)
    f1 = f1_score(y_te, y_pred)
    cm = confusion_matrix(y_te, y_pred)
    print(f"\n{title} Metrics:\nAccuracy: {acc:.4f}\nPrecision: {prec:.4f}\nRecall: {rec:.4f}\nF1: {f1:.4f}")
    sns.heatmap(cm, annot=True, fmt='d')
    plt.title(f'Confusion Matrix - {title}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    return {'accuracy':acc, 'precision':prec, 'recall':rec, 'f1':f1}

metrics_baseline_lr = eval_model(baseline_lr, X_train, y_train, X_test, y_test, 'LogisticRegression (All Features)')
metrics_baseline_rf = eval_model(baseline_rf, X_train, y_train, X_test, y_test, 'RandomForest (All Features)')

## Step 4B: Feature Selection
We will apply **two** methods and pick the better one using cross-validation on the training set:
- **Filter**: SelectKBest with ANOVA F-test (`f_classif`)
- **Wrapper**: RFE with Logistic Regression estimator

We'll search for the best **k** for SelectKBest among a small set to keep it simple (no hyperparameter tuning of the classifier).

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
k_values = [5, 10, 15, 20, 25, 30]
best_k = None
best_cv_score = -np.inf

for k in k_values:
    pipe = Pipeline([
        ('select', SelectKBest(score_func=f_classif, k=k)),
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
    ])
    scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='accuracy')
    mean_score = scores.mean()
    print(f'k={k}: CV Accuracy={mean_score:.4f}')
    if mean_score > best_cv_score:
        best_cv_score = mean_score
        best_k = k

print(f"\nBest k by CV: {best_k} (CV Acc={best_cv_score:.4f})")

In [ ]:
# Build final SelectKBest selector using best_k
selector_kbest = SelectKBest(score_func=f_classif, k=best_k)
selector_kbest.fit(X_train, y_train)
selected_mask = selector_kbest.get_support()
selected_features_kbest = X_train.columns[selected_mask].tolist()
print('Selected features (SelectKBest):', selected_features_kbest)

In [ ]:
# Wrapper method: RFE with Logistic Regression
estimator = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
rfe = RFE(estimator=estimator, n_features_to_select=best_k)
rfe.fit(X_train, y_train)
selected_features_rfe = X_train.columns[rfe.support_].tolist()
print('Selected features (RFE):', selected_features_rfe)

# Compare which set performs better via CV with LR
def cv_score_for_feature_set(features):
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
    ])
    return cross_val_score(pipe, X_train[features], y_train, cv=cv, scoring='accuracy').mean()

cv_kbest = cv_score_for_feature_set(selected_features_kbest)
cv_rfe = cv_score_for_feature_set(selected_features_rfe)
print(f'CV Acc (SelectKBest features): {cv_kbest:.4f}')
print(f'CV Acc (RFE features): {cv_rfe:.4f}')

if cv_rfe > cv_kbest:
    selected_features = selected_features_rfe
    fs_method = 'RFE'
else:
    selected_features = selected_features_kbest
    fs_method = 'SelectKBest'

print(f"Chosen feature set from: {fs_method}")

## Step 5: Train Models **With Feature Selection** (using chosen features)
We'll retrain both classifiers using only the selected features.

In [ ]:
with_fs_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])
with_fs_rf = Pipeline([
    ('clf', RandomForestClassifier(random_state=RANDOM_STATE))
])

with_fs_lr.fit(X_train[selected_features], y_train)
with_fs_rf.fit(X_train[selected_features], y_train)

metrics_withfs_lr = eval_model(with_fs_lr, X_train[selected_features], y_train, X_test[selected_features], y_test, f'LogisticRegression (Selected features: {fs_method})')
metrics_withfs_rf = eval_model(with_fs_rf, X_train[selected_features], y_train, X_test[selected_features], y_test, f'RandomForest (Selected features: {fs_method})')

## Step 6: Save Two Models as `.pkl`
Per deliverables, we will save two pipelines based on **Logistic Regression**:
- `model_without_fs.pkl` (StandardScaler + LogisticRegression on **all** features)
- `model_with_fs.pkl` (Column selection → StandardScaler + LogisticRegression on **selected** features)

To keep inference easy in Streamlit, we save a **single pipeline** that internally selects columns by name before scaling.


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

all_features = X_train.columns.tolist()

def select_columns(X_df, cols):
    # X_df will be a numpy array inside pipeline; convert if needed
    if isinstance(X_df, pd.DataFrame):
        return X_df[cols]
    else:
        # if array, convert to DataFrame first
        return pd.DataFrame(X_df, columns=all_features)[cols]

# Pipeline WITHOUT FS (uses all features)
pipeline_without_fs = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])
pipeline_without_fs.fit(X_train, y_train)
joblib.dump({'pipeline': pipeline_without_fs, 'feature_names': all_features}, 'model_without_fs.pkl')

# Pipeline WITH FS (select columns by name, then scale + LR)
select_transformer = FunctionTransformer(lambda df: select_columns(df, selected_features))
pipeline_with_fs = Pipeline([
    ('select', select_transformer),
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])
pipeline_with_fs.fit(X_train, y_train)
joblib.dump({'pipeline': pipeline_with_fs, 'feature_names': all_features, 'selected_features': selected_features, 'method': fs_method}, 'model_with_fs.pkl')

print('Saved model_without_fs.pkl and model_with_fs.pkl to current working directory.')

## Step 7: Compare Before vs After (Table)


In [ ]:
comparison = pd.DataFrame([
    {'Model':'LogReg-All', **metrics_baseline_lr},
    {'Model':f'LogReg-{fs_method}', **metrics_withfs_lr},
    {'Model':'RF-All', **metrics_baseline_rf},
    {'Model':f'RF-{fs_method}', **metrics_withfs_rf},
])
display(comparison)


## Step 8: Short Explanation (edit this cell)

**How optimization changed performance & why:**
- Feature selection removed noisy/redundant features, which can reduce variance and improve generalization.
- Logistic Regression often benefits from a smaller, informative subset (better coefficients, less overfitting).
- Random Forest is robust to redundant features, so changes may be smaller.
- If accuracy decreased after selection, likely the chosen **k** was too small or removed weak-but-useful features.
- Our chosen method: **{{fs_method}}** with **k={{best_k}}** produced the best CV accuracy on the training set.
